In [ ]:
#| hide
from faststripe.core import *

# faststripe

> Fastest way to use the Stripe API in python

## Why FastStripe?

FastStripe offers several advantages over the official Stripe Python SDK:

- **🔍 Self-documenting**: See all available parameters with descriptions in your IDE
- **🎯 Simplified workflows**: High-level methods for common payment patterns
- **📦 Lightweight**: Minimal dependencies (just `httpx` and `fastcore`)
- **🏗️ Consistent API**: Uniform `create`/`fetch` naming across all resources
- **💫 AttrDict responses**: Access response data with dot notation (`customer.email`)
- **🔧 Automatic data handling**: Nested dictionaries flattened for Stripe's API format

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/AnswerDotAI/faststripe.git
```

or from [pypi][pypi]


```sh
$ pip install faststripe
```


[repo]: https://github.com/AnswerDotAI/faststripe
[docs]: https://AnswerDotAI.github.io/faststripe/
[pypi]: https://pypi.org/project/faststripe/
[conda]: https://anaconda.org/AnswerDotAI/faststripe

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/AnswerDotAI/faststripe
[docs]: https://AnswerDotAI.github.io/faststripe/
[pypi]: https://pypi.org/project/faststripe/
[conda]: https://anaconda.org/AnswerDotAI/faststripe

## How to use

FastStripe provides a simple, self-documenting interface to the Stripe API. All you need is your Stripe API key:

In [ ]:
from faststripe.core import StripeApi

import os

# Initialize with your API key from environment
sapi = StripeApi(os.environ['STRIPE_SECRET_KEY'])

In [ ]:
# Create a customer
customer = sapi.customers.create(email='user@example.com', name='John Doe')
print(customer.id, customer.email)

cus_SMsM453Vb64XcF user@example.com


### Self-Documenting API

One of FastStripe's key advantages is that all methods include parameter documentation directly in your IDE. You can see what parameters are available without checking external docs:

In [ ]:
# Explore available methods and their parameters
sapi.customers.create?

Signature:
sapi.customers.create(
    *,
    address=None,
    balance=None,
    cash_balance=None,
    description=None,
    email=None,
    expand=None,
    invoice_prefix=None,
    invoice_settings=None,
    metadata=None,
    name=None,
    next_invoice_sequence=None,
    payment_method=None,
    phone=None,
    preferred_locales=None,
    shipping=None,
    source=None,
    tax=None,
    tax_exempt=None,
    tax_id_data=None,
    test_clock=None,
)
Docstring:
Create a customer

Parameters:
    address: The customer's address.
    balance: An integer amount in cents (or local equivalent) that represents the customer's current balance, which affect the customer's future invoices. A negative amount represents a credit that decreases the amount due on an invoice; a positive amount increases the amount due on an invoice.
    cash_balance: Balance information and default balance settings for this customer.
    description: An arbitrary string that you can attach to a customer object. It

### High-Level Convenience Methods

FastStripe includes simplified methods for common payment workflows:

In [ ]:
# Create a one-time payment checkout session
checkout = sapi.one_time_payment(
    product_name='My Product',
    amount_cents=2000,  # $20.00
    success_url='https://localhost:8000/success',
    cancel_url='https://localhost:8000/cancel'
)
print(f"Payment URL: {checkout.url[:64]}...")

Payment URL: https://billing.answer.ai/c/pay/cs_test_a17Swg1TqARRl0ItVDkdelkR...


In [ ]:
# Create a subscription checkout session
subscription = sapi.subscription(
    product_name='Monthly Plan',
    amount_cents=999,  # $9.99/month
    success_url='https://localhost:8000/success',
    cancel_url='https://localhost:8000/cancel',
    customer_email=customer.email
)
print(f"Subscription URL: {subscription.url[:64]}...")

Subscription URL: https://billing.answer.ai/c/pay/cs_test_a1xQzvl0Qd6arXt9EKWsYDJ7...


### Complete API Coverage

FastStripe provides access to the entire Stripe API through organized resource groups:

In [ ]:
# Access any Stripe resource with consistent patterns
product = sapi.products.create(name='New Product')
print(f"Created product: {product.name} with ID: {product.id}")

Created product: New Product with ID: prod_SMsMg2mD205PNM


In [ ]:
# Fetch existing resources
customers = sapi.customers.fetch(limit=3)
print(f"Found {len(customers.data)} customers")

Found 3 customers


In [ ]:
# All responses are AttrDict objects for easy dot notation access
payment_intent = sapi.payment_intents.create(amount=1000, currency='usd')
print(f"Payment intent status: {payment_intent.status}, amount: ${payment_intent.amount/100}")

Payment intent status: requires_payment_method, amount: $10.0
